<a href="https://colab.research.google.com/github/EssenceBL/MakingFriends/blob/main/Study_Notes_C_Extension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Foreword**

This series of Colab notebooks contains study notes related to Descartes' Circle Theorem. The motivation for this work originated from attending Professor Peter Sarnak's Shaw Prize Lecture in Mathematical Sciences in 2024. Not only does his lecture begin with Descartes' Circle Theorem, but he also provides significant insights into the importance of establishing a "base" for one’s long-term endeavors. Reflecting on his remarks, I believe that exploring mathematical concepts through Python code could serve as my "base," drawing from prior experiences; thus, I produced this series of Colab notebooks. Some of the code comes from iterative collaboration with DeepSeek, representing an experiment that integrates traditional mathematical mindsets with contemporary computational methodologies.

YK with DeepSeek, 2025

Like other Colab notebooks, please run the code cells one by one. In particular, the following code cell concerns the dependency packages and needs to be run first. It might take a while, so please be patient.

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import cmath
import math

**An Interactive Game**

Just try it out with your friends~

In [ ]:
class Circle:
  def __init__(self, curv, center):
    self.curv = curv
    self.center = center  # complex number
    self.radius = 1 / abs(curv) if curv != 0 else float('inf')
  def __eq__(self, other):
    tolerance = 1e-5
    return (abs(self.curv - other.curv) < tolerance and
            abs(self.center - other.center) < tolerance)
  def __hash__(self):
    return hash((self.curv, self.center.real, self.center.imag))

def compute_fourth_curvature(b1, b2, b3):
  sum_b = b1 + b2 + b3
  root_term = 2 * math.sqrt(abs(b1*b2 + b2*b3 + b3*b1))
  return sum_b + root_term, sum_b - root_term

def compute_fourth_circle(c1, c2, c3, b4):
  b1, b2, b3 = c1.curv, c2.curv, c3.curv
  z1, z2, z3 = c1.center, c2.center, c3.center

  sum_b = b1 + b2 + b3
  root_term = 2 * math.sqrt((b1*b2 + b2*b3 + b3*b1))
  b4_plus = sum_b + root_term
  b4_minus = sum_b - root_term
  #b4 = min(b4_plus, b4_minus, key=lambda x: abs(x))

  S = b1*z1 + b2*z2 + b3*z3
  Q = cmath.sqrt(b1*b2*z1*z2 + b1*b3*z1*z3 + b2*b3*z2*z3)
  z4_plus = (S + 2*Q) / b4_plus
  z4_minus = (S - 2*Q) / b4_minus
  # Check which solution is valid (inside the outer circle)
  candidates = []
  candidates.append(Circle(b4_plus, z4_plus))
  candidates.append(Circle(b4_minus, z4_minus))
  z4_plus = (S - 2*Q) / b4_plus
  z4_minus = (S + 2*Q) / b4_minus
  candidates.append(Circle(b4_plus, z4_plus))
  candidates.append(Circle(b4_minus, z4_minus))
  return candidates

def apollonian(c1, c2, c3, c4, g):
  circles = [c1, c2, c3, c4]
  i = 'abcd'.index(g)
  b = [circles[i].curv for i in range(4)]
  bz = [circles[i].curv*circles[i].center for i in range(4)]
  ci_new_curv = 2*(sum(b))-3*circles[i].curv
  ci_new_center = (2*(sum(bz))-3*circles[i].curv*circles[i].center)/ci_new_curv
  ci_new = Circle(ci_new_curv, ci_new_center)
  circles[i] = ci_new
  return circles[0], circles[1], circles[2], circles[3]

class ApollonianWordTuple:
  def __init__(self, fourTuple, initial = 'aab', generators=None):
    self.fourTuple = fourTuple  # No default to avoid NameError
    self.initial = initial
    self.generators = generators.copy() if generators else ['a', 'b', 'c', 'd']
    # Each generator is its own inverse (modify if needed)
    self.inverse = {g: g for g in self.generators}
    self.generations = {}  # Cache for generations

  def get_generation(self, i):
    if i in self.generations:
      return self.generations[i]

    if i < 0:
      raise ValueError("Generation index cannot be negative")

    # Base case: generation 0 is the initial fourTuple
    if i == 0:
      gen = [["", self.fourTuple]]
      self.generations[0] = gen
      return gen

    # Recursive case: build from previous generation
    prev_gen = self.get_generation(i - 1)
    current_gen = []
    for word, circles in prev_gen:
      last_char = word[-1] if word else None
      forbidden = self.inverse.get(last_char, None)
      if len(self.initial) > i-1:
        # Assuming apollonian function returns new circle configuration
        new_circles = apollonian(
            circles[0], circles[1], circles[2], circles[3], self.initial[i-1]
        )
        current_gen.append([word + self.initial[i-1], new_circles])
        self.generations[i] = current_gen
        return current_gen
      for g in self.generators:
        if g != forbidden:
          new_circles = apollonian(
            circles[0], circles[1], circles[2], circles[3], g
          )
          current_gen.append([word + g, new_circles])
    self.generations[i] = current_gen
    return current_gen

def plot_circles(circles, ax):
  for [circle, word] in circles:
    if circle.curv == 0:
      continue
    radius = circle.radius
    center = circle.center
    color = 'red' if word[-1] == 'a' else 'orange' if word[-1] == 'b' else 'pink' if word[-1] == 'c' else 'blue'
    circ = plt.Circle((center.real, center.imag), radius, fill=False, color=color, linewidth=2 if circle.curv > 0 else 2)
    if radius > 9e-5:
      ax.add_patch(circ)

def plot_generated_circles(b1 = 3, b2 = 6, b3 = 7, initial = '', generations = 5):
  b4_plus, b4_minus = compute_fourth_curvature(b1, b2, b3)
  b4 = min(b4_plus, b4_minus, key=lambda x: abs(x))  # Choose enclosing circle (negative)
  z1 = 0 + 0j
  z2 = 1/b1+1/b2 + 0j
  x = (1/(b1*b1)+1/(b1*b2)+1/(b1*b3)-1/(b2*b3))/(1/b1+1/b2)
  z3 = x - cmath.sqrt(-(1/b1+1/b3)*(1/b1+1/b3)+x*x)
  b4_plus = compute_fourth_curvature(b1, b2, b3)[0]
  b4_minus = compute_fourth_curvature(b1, b2, b3)[1]
  S = b1*z1 + b2*z2 + b3*z3
  Q = cmath.sqrt(b1*b2*z1*z2 + b1*b3*z1*z3 + b2*b3*z2*z3)
  z4_plus = (S + 2*Q) / b4_plus
  z4_minus = (S - 2*Q) / b4_minus
  # Initial circles
  circles = [[Circle(b1, z1), 'aa'], [Circle(b2, z2), 'bb'], [Circle(b3, z3), 'cc'], [Circle(b4_minus, z4_minus), 'dd']]
  drawApollonian = ApollonianWordTuple(fourTuple = [circles[i][0] for i in range(4)], initial=initial)
  # Plot
  fig, ax = plt.subplots(figsize=(8, 8))
  for [circle, word] in circles:
    color = 'red' if word[-1] == 'a' else 'orange' if word[-1] == 'b' else 'pink' if word[-1] == 'c' else 'blue'
    plt.text(circle.center.real, circle.center.imag, f"{circle.curv:.1f}", ha='center', va='center', color=color, fontsize=b2*30/circle.curv)
  for i in range(1,len(initial)+generations):
    for word, Tuple in drawApollonian.get_generation(i):
      circle = Tuple['abcd'.index(word[-1])]
      circles.append([circle, word])
      if circle.radius > 9e-5:
        if word[-1] == 'a':
          plt.text(circle.center.real, circle.center.imag, f"{circle.curv:.1f}", ha='center', va='center', color='red', fontsize=b2*30/circle.curv)
        if word[-1] == 'b':
          plt.text(circle.center.real, circle.center.imag, f"{circle.curv:.1f}", ha='center', va='center', color='orange', fontsize=b2*30/circle.curv)
        if word[-1] == 'c':
          plt.text(circle.center.real, circle.center.imag, f"{circle.curv:.1f}", ha='center', va='center', color='pink', fontsize=b2*30/circle.curv)
        if word[-1] == 'd':
          plt.text(circle.center.real, circle.center.imag, f"{circle.curv:.1f}", ha='center', va='center', color='blue', fontsize=b2*30/circle.curv)
        #if len(word) <= len(initial):
        #  plt.text(circle.center.real, circle.center.imag, f"{circle.curv:.1f}", ha='center', va='center', color='blue', fontsize=b2*30/circle.curv)
        #else:
        #  plt.text(circle.center.real, circle.center.imag, f"{circle.curv:.1f}", ha='center', va='center', color='blue', fontsize=b2*30/circle.curv)
  ax.set_aspect('equal')
  ax.set_title("An Interactive Game")
  ax.set_xlim(z4_minus.real-1/b4_minus+1/b4_plus, z4_minus.real+1/b4_minus-1/b4_plus)
  ax.set_ylim(z4_minus.imag-1/b4_minus+1/b4_plus, z4_minus.imag+1/b4_minus-1/b4_plus)
  plot_circles(circles, ax)
  plt.show()

def plot(input_str, current):
  try:
    plot_generated_circles(b1 = 2, b2 = 2, b3 = 3, initial = input_str, generations = int(current or 0))
  except Exception as e:
    print(f"Error: {str(e)}")

input_widget = widgets.Text(
    value='',
    #value='aabbccd',
    #value='bcbcbcbcbcbccbcbcbcbcbcbcbcbcbcbc',
    placeholder='consists of a, b, c or d (e.g. acacac)',
    description='Sequence of circles:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='380px')
)

input_widget_2 = widgets.Text(
    value='5',
    placeholder='Enter a positive integer (e.g. 1, 2, 3)',
    description='Generation expansion from the last one:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='380px')
)

widgets.interactive(plot, input_str=input_widget, current=input_widget_2)

interactive(children=(Text(value='', description='Sequence of circles:', layout=Layout(width='380px'), placeho…